In [ ]:
### import manually defined automata
%matplotlib inline
from csrl.pomdp import GridPOMDP
from csrl.oaa import oaa
from csrl import ControlSynthesis
import numpy as np 

oa=oaa()

print('Number of Omega-automaton states (including the trap state):',oa.shape[1])
display(oa)

print('Initial state:',oa.q0)
print('Transition function: ['),print(*['  '+str(t) for t in oa.delta],sep=',\n'),print(']')
print('Acceptance: ['),print(*['  '+str(t) for t in oa.acc],sep=',\n'),print(']')

In [ ]:
# POMDP Description
shape = (10,10)
# E: Empty, T: Trap, B: Obstacle
structure = np.array([
['E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E'],
['E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E'],
['E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E'],
['E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E'],
['E',  'E',  'E',  'E',  'B',  'B',  'E',  'E',  'E',  'E'],
['E',  'E',  'E',  'E',  'B',  'B',  'E',  'E',  'E',  'E'],
['E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E'],
['E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E'],
['E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E'],
['E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E',  'E']
])

# Labels of the states
label = np.array([
[(),       (),       (),       (),       (),       (),       (),       (),       ('b',),       ('b',)],
[(),       (),       (),       (),       (),       (),       (),       (),       ('b',),       ('b',)],
[(),       (),       ('c',),       ('c',),       (),       (),       (),       (),       (),       ()],
[(),       (),       ('c',),       ('c',),       (),       (),       (),       (),       (),       ()],
[(),       (),       (),       (),       (),       (),       (),       (),       (),       ()],
[(),       (),       (),       (),       (),       (),       (),       (),       (),       ()],
[(),       (),       (),       (),       (),       (),       ('c',),       ('c',),       (),       ()],
[(),       (),       (),       (),       (),       (),       ('c',),       ('c',),       (),       ()],
[('a',),       ('a',),       (),       (),       (),       (),       (),       (),       (),       ()],
[('a',),       ('a',),       (),       (),       (),       (),       (),       (),       (),       ()]
],dtype=np.object)

grid_pomdp = GridPOMDP(shape=shape,structure=structure,label=label)

# Construct the product POMDP
csrl = ControlSynthesis(grid_pomdp,oa)

In [ ]:
csrl.belief_state

In [ ]:
csrl.A

In [ ]:
csrl.reward

In [ ]:
csrl.train_DQN(EPISODES=10000,num_steps=500)

In [ ]:
################### verify the dqn_cnn policy ###################

from dqn_cnn import DQNAgent
EPISODES=10
num_steps=100

Path = []

gamma=0.99999
gammaB=0.9

# the defined belief_state size and action size
belief_state_size = np.shape(csrl.belief_state)

# find the size of belief_state for np.reshape
prod_b_state_size = 1
for i in range(len(belief_state_size)):
    prod_b_state_size = prod_b_state_size * belief_state_size[i]

# action size
#prod_action_size = csrl.shape[4]
prod_action_size = 4

agent = DQNAgent(prod_b_state_size, csrl.shape[2], csrl.shape[3], csrl.shape[1], prod_action_size, gamma, gammaB)
agent.load("./save/DQN_CNN_10_frontier.h5")
agent.epsilon = 0
done = False
num_episode_for_reward = 100 # print the accumulated reward per num of episode
# initialize the list for plot
accumulated_rewards=[]
exploration_rate=[]
accumulated_rewards_hundred_steps = []

for e in range(EPISODES):
    accumulated_rewards_per_episode=0
    done = False
    subpath = []

    pomdp_state = csrl.pomdp.random_state()
    while csrl.pomdp.label[pomdp_state[0],pomdp_state[1]] == ('c',) or csrl.pomdp.structure[(pomdp_state[0],pomdp_state[1])]=='B':
        pomdp_state = csrl.pomdp.random_state()
    pomdp_state = (9, 7) # edit here to specify the start state or comment it out
    state = (csrl.shape[0]-1,csrl.oa.q0)+(pomdp_state) # select the start product state
    print('START STATE: '+str(state))
    belief_state = csrl.belief_state # initialize the belief state
    csrl.track = [0,1] #self.initial_track # initialize frontier set = [0,1]
    reshaped_reward = csrl.reshaped_reward_init

    for step in range(num_steps):
        print(state)
        subpath.append(state)
        # reshape the belief state as the input to acquire the action
        input_b_state = np.reshape(belief_state,(1, csrl.shape[2], csrl.shape[3], csrl.shape[1]))
        print('state: '+str(state))
        # verify the existence of action and select the action from the belief_state
        action_probs = agent.act_trained(input_b_state)
        action_probs = np.reshape(action_probs,(prod_action_size,1))
        i = 0
        possible_actions = []
        for i in range(len(csrl.A[state])):
            possible_actions.append(action_probs[csrl.A[state][i]])
        action = csrl.A[state][np.argmax(possible_actions)]
        print('action selected: '+str(action))

        ################## POMDP simualtion ##################

        # agent moves to the next state
        states, probs = csrl.transition_probs[state][action]
        next_state = states[np.random.choice(len(states),p=probs)]
        # find the observation states' list and the corresponding probabilities
        obsv_states, obsv_probs = csrl.pomdp.get_observation_prob(next_state[-2:])
        # observe the next state
        obsv_state = csrl.pomdp.generate_obsv_state(obsv_states, obsv_probs)

        ################## The belief_state update with the loops ##################

        # temproraily store the current belief state
        current_belief_state = belief_state
        # belief state update
        belief_state_after_transition = []
        for s in csrl.states():
            belief_state_after_transition.append(belief_state[s]*csrl.belief_transition_probs[s][action])
        belief_state_after_transition = sum(belief_state_after_transition)
        # update the belief state with the observation probability matrix
        updated_belief_state = belief_state_after_transition
        for s in csrl.states():
            updated_belief_state[s] = updated_belief_state[s]*csrl.belief_observation_probs[s][obsv_state[0], obsv_state[1]]
        belief_state = updated_belief_state/sum(sum(sum(sum(updated_belief_state))))

        ################## Test the DQN model ##################
        
        # Update the frontier set + update reward setup accordingly
        reshaped_reward = csrl.Tf(state, next_state, reshaped_reward)
        # calculate the reward from the next belief state and find gamm
        reward = np.sum(np.reshape(belief_state,(1,prod_b_state_size))*reshaped_reward)
        # assign the next state
        state = next_state
        # record the accumulated rewards
        accumulated_rewards_per_episode = accumulated_rewards_per_episode + reward
        print('reward: '+str(reward))
    
    Path.append(subpath) # append the apth
    
    print('accumulated_rewards_per_episode: '+str(accumulated_rewards_per_episode))
    accumulated_rewards.append(accumulated_rewards_per_episode)
    exploration_rate.append(agent.epsilon)

    if len(accumulated_rewards)>=num_episode_for_reward:
        accumulated_rewards_hundred_steps.append(np.average(accumulated_rewards))
        accumulated_rewards = []

import matplotlib.pyplot as plt
t1 = np.arange(0, EPISODES, 1)
t2 = np.arange(0, len(accumulated_rewards_hundred_steps)*num_episode_for_reward, num_episode_for_reward)

plt.figure(figsize=(14, 6))
display(plt.plot(t1, exploration_rate))

plt.figure(figsize=(14, 6))
display(plt.plot(t2, accumulated_rewards_hundred_steps))

In [ ]:
############### Plot the path ###############
import pylab as pl
from matplotlib.collections import LineCollection

size_x = 10
size_y = 10

In [ ]:
path = Path[1]

### plot the path on 'q0'.

x = np.linspace(0,size_x,size_x+1)
y = np.linspace(0,size_y,size_y+1) 

pl.figure(figsize=(10,10))

hlines = np.column_stack(np.broadcast_arrays(x[0], y, x[-1], y))
vlines = np.column_stack(np.broadcast_arrays(x, y[0], x, y[-1]))
lines = np.concatenate([hlines, vlines]).reshape(-1, 2, 2)
line_collection = LineCollection(lines, color="black", linewidths=1)
ax = pl.gca()
ax.add_collection(line_collection)
ax.set_xlim(int(x[0]), int(x[-1]))
ax.set_ylim(int(y[0]), int(y[-1]))
# backgroud color
ax.add_patch(pl.Rectangle((0, 0), size_x, size_y, fill=True, color='green', alpha=.1))

# plot the Blocks
b_start_x = b_start_y = 4
b_size_x = b_size_y = 2
ax.add_patch(pl.Rectangle((b_start_x, b_start_y), b_size_x, b_size_y, fill=True, color='black', alpha=.5))

# plot the Traps, 'c'
b_start_x = 2
b_start_y = 6
b_size_x = b_size_y = 2
ax.add_patch(pl.Rectangle((b_start_x, b_start_y), b_size_x, b_size_y, fill=True, color='orange', alpha=.8))

b_start_x = 6
b_start_y = 2
b_size_x = b_size_y = 2
ax.add_patch(pl.Rectangle((b_start_x, b_start_y), b_size_x, b_size_y, fill=True, color='orange', alpha=.8))

# plot the 'a's
b_start_x = b_start_y = 0
b_size_x = b_size_y = 2
ax.add_patch(pl.Rectangle((b_start_x, b_start_y), b_size_x, b_size_y, fill=True, color='blue', alpha=.5))

# plot the 'b's
b_start_x = b_start_y = 8
b_size_x = b_size_y = 2
ax.add_patch(pl.Rectangle((b_start_x, b_start_y), b_size_x, b_size_y, fill=True, color='green', alpha=.5))    
        
for i in range(len(path)):
    if path[i][1]<=0:
        state_idx = path[i]
        ## convert state index in Julia to 'x,y' coordinates in python
        
        coord_x = path[i][3]
        coord_y = size_x-1-path[i][2]
        
        ax.add_patch(pl.Circle((coord_x+0.5, coord_y+0.5), 0.2, fill=True, color='red', alpha=.4))
        if i==0:
            # start point
            ax.add_patch(pl.Circle((coord_x+0.5, coord_y+0.5), 0.4, fill=True, color='purple', alpha=.9))
    else:
        break

In [ ]:
### plot the path on 'q1'.

x = np.linspace(0,size_x,size_x+1)
y = np.linspace(0,size_y,size_y+1) 

pl.figure(figsize=(10,10))

hlines = np.column_stack(np.broadcast_arrays(x[0], y, x[-1], y))
vlines = np.column_stack(np.broadcast_arrays(x, y[0], x, y[-1]))
lines = np.concatenate([hlines, vlines]).reshape(-1, 2, 2)
line_collection = LineCollection(lines, color="black", linewidths=1)
ax = pl.gca()
ax.add_collection(line_collection)
ax.set_xlim(int(x[0]), int(x[-1]))
ax.set_ylim(int(y[0]), int(y[-1]))
# backgroud color
ax.add_patch(pl.Rectangle((0, 0), size_x, size_y, fill=True, color='green', alpha=.1))

# plot the Blocks
b_start_x = b_start_y = 4
b_size_x = b_size_y = 2
ax.add_patch(pl.Rectangle((b_start_x, b_start_y), b_size_x, b_size_y, fill=True, color='black', alpha=.5))

# plot the Traps, 'c'
b_start_x = 2
b_start_y = 6
b_size_x = b_size_y = 2
ax.add_patch(pl.Rectangle((b_start_x, b_start_y), b_size_x, b_size_y, fill=True, color='orange', alpha=.8))

b_start_x = 6
b_start_y = 2
b_size_x = b_size_y = 2
ax.add_patch(pl.Rectangle((b_start_x, b_start_y), b_size_x, b_size_y, fill=True, color='orange', alpha=.8))

# plot the 'a's
b_start_x = b_start_y = 0
b_size_x = b_size_y = 2
ax.add_patch(pl.Rectangle((b_start_x, b_start_y), b_size_x, b_size_y, fill=True, color='blue', alpha=.5))

# plot the 'b's
b_start_x = b_start_y = 8
b_size_x = b_size_y = 2
ax.add_patch(pl.Rectangle((b_start_x, b_start_y), b_size_x, b_size_y, fill=True, color='green', alpha=.5))    

ii=i
for i in range(ii, len(path)):
    if path[i][1]==1:
        state_idx = path[i]
        ## convert state index in Julia to 'x,y' coordinates in python
        coord_x = path[i][3]
        coord_y = size_x-1-path[i][2]
        
        ax.add_patch(pl.Circle((coord_x+0.5, coord_y+0.5), 0.2, fill=True, color='red', alpha=.4))
        if i==0:
            # start point
            ax.add_patch(pl.Circle((coord_x+0.5, coord_y+0.5), 0.4, fill=True, color='purple', alpha=.9))
    else:
        break

In [ ]:
### plot again the path on 'q0'.

x = np.linspace(0,size_x,size_x+1)
y = np.linspace(0,size_y,size_y+1) 

pl.figure(figsize=(10,10))

hlines = np.column_stack(np.broadcast_arrays(x[0], y, x[-1], y))
vlines = np.column_stack(np.broadcast_arrays(x, y[0], x, y[-1]))
lines = np.concatenate([hlines, vlines]).reshape(-1, 2, 2)
line_collection = LineCollection(lines, color="black", linewidths=1)
ax = pl.gca()
ax.add_collection(line_collection)
ax.set_xlim(int(x[0]), int(x[-1]))
ax.set_ylim(int(y[0]), int(y[-1]))
# backgroud color
ax.add_patch(pl.Rectangle((0, 0), size_x, size_y, fill=True, color='green', alpha=.1))

# plot the Blocks
b_start_x = b_start_y = 4
b_size_x = b_size_y = 2
ax.add_patch(pl.Rectangle((b_start_x, b_start_y), b_size_x, b_size_y, fill=True, color='black', alpha=.5))

# plot the Traps, 'c'
b_start_x = 2
b_start_y = 6
b_size_x = b_size_y = 2
ax.add_patch(pl.Rectangle((b_start_x, b_start_y), b_size_x, b_size_y, fill=True, color='orange', alpha=.8))

b_start_x = 6
b_start_y = 2
b_size_x = b_size_y = 2
ax.add_patch(pl.Rectangle((b_start_x, b_start_y), b_size_x, b_size_y, fill=True, color='orange', alpha=.8))

# plot the 'a's
b_start_x = b_start_y = 0
b_size_x = b_size_y = 2
ax.add_patch(pl.Rectangle((b_start_x, b_start_y), b_size_x, b_size_y, fill=True, color='blue', alpha=.5))

# plot the 'b's
b_start_x = b_start_y = 8
b_size_x = b_size_y = 2
ax.add_patch(pl.Rectangle((b_start_x, b_start_y), b_size_x, b_size_y, fill=True, color='green', alpha=.5))    

ii=i
for i in range(ii, len(path)):
    if path[i][1]<=0:
        state_idx = path[i]
        ## convert state index in Julia to 'x,y' coordinates in python
        coord_x = path[i][3]
        coord_y = size_x-1-path[i][2]
        
        ax.add_patch(pl.Circle((coord_x+0.5, coord_y+0.5), 0.2, fill=True, color='red', alpha=.4))
        if i==0:
            # start point
            ax.add_patch(pl.Circle((coord_x+0.5, coord_y+0.5), 0.4, fill=True, color='purple', alpha=.9))
    else:
        break